# Dogs vs Cats 데이터셋

[Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition) 데이터셋에는 훈련용으로 고양이, 개 사진이 각각 12,500장 그리고 테스트용으로 총 12,500장이 있습니다.  
이번 실습의 목표는 적은 수의 데이터만 주어져 있을 때 데이터 증강(data augmentation)을 통해 신경망의 성능을 끌어올리는 것 입니다.  
따라서, 전체 데이터 셋을 사용하지 않고 아주 일부분만을 사용하겠습니다.  
![](https://drive.google.com/thumbnail?id=1IwvrXV6LNsedoxM5iOrfK5vlnUGw0qyG&sz=s4000)

실습에 사용할 일부분을 미리 추려내서 압축한 후 구글 드라이브에 업로드했습니다.  
`gdown` 라이브러리를 사용하면 구글 드라이브에 링크된 파일을 다운로드 받을 수 있습니다. (프롬프트에서 `pip install gdown`으로 설치)  
`os.path.isdir('cats_vs_dogs_small')`은 현재 작업디렉토리에 cats_vs_dogs_small란 이름의 디렉토리가 있으면 True를 리턴합니다.  
`zipfile` 라이브러리의 `extractall()`을 써서 압축파일의 하위 디렉토리까지 모두 풀어줍니다.  
파일탐색기로 현재 작업디렉토리 밑에 cats_vs_dogs_small 디렉토리, 그 밑에 test, train, validation 디렉토리, 그밑에 각각 cat, dog 디렉토리가 생성된 걸 확인할 수 있습니다.

In [ ]:
!pip install gdown

In [ ]:
import gdown, zipfile, os

if not os.path.isdir('cats_vs_dogs_small'):
    gdown.download(id='1z2WPTBUI-_Q2jZtcRtQL0Vxigh-z6dyW', output='cats_vs_dogs_small.zip')
    cats_vs_dogs_small = zipfile.ZipFile('cats_vs_dogs_small.zip')
    cats_vs_dogs_small.extractall()
    cats_vs_dogs_small.close()

훈련용 고양이 디렉토리 안에는 다음과 같은 이름의 파일들이 있습니다.

In [ ]:
print(sorted(os.listdir("./cats_vs_dogs_small/train/cat")))

개와 고양이 사진이 각각 훈련용 1,000장, 검증용 500장, 테스트용 1,000장이 있습니다.

In [ ]:
print("train : "+sorted(os.listdir("./cats_vs_dogs_small/train/cat"))[0]+" ~ "+sorted(os.listdir("./cats_vs_dogs_small/train/cat"))[-1])
print("validation : "+sorted(os.listdir("./cats_vs_dogs_small/validation/cat"))[0]+" ~ "+sorted(os.listdir("./cats_vs_dogs_small/validation/cat"))[-1])
print("test : "+sorted(os.listdir("./cats_vs_dogs_small/test/cat"))[0]+" ~ "+sorted(os.listdir("./cats_vs_dogs_small/test/cat"))[-1])

고양이 첫 훈련 이미지 25장입니다.  
해상도가 제각각이고 사람과 찍은 사진, 여러마리 사진, 철창에 가려진 사진도 보이네요.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as image

plt.figure(figsize=(15,15))
for i in range(25):
    plt.subplot(5,5,i+1)
    img_path = f'./cats_vs_dogs_small/train/cat/cat.{i}.jpg'
    img = image.imread(img_path)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
plt.show()

**[실습1] (2분) 강아지 첫 훈련 이미지 25장을 5$\times$5 모아찍기로 출력하시오.**

# 입력 파이프라인 API

리스트와 튜플은 순회 가능한 객체(iterable object)입니다.  
사실 반복문에서 `in` 다음에 오는 것은 모두 순회 가능한 객체입니다.  
순회 가능한 객체는 `iter`를 씌운 후 `next`를 적용하면 순서대로 내장된 값을 리턴합니다.

In [ ]:
x = ["a","b","c"]
iterator = iter(x)
print(next(iterator))
print(next(iterator))
print(next(iterator))

**[실습2] (2분) `range(3)`에 `iter`를 씌운 후 `next`를 차례로 적용하시오.**

텐서플로우는 대용량 데이터를 다룰때 통상적으로 [tf.data](https://www.tensorflow.org/api_docs/python/tf/data) API를 이용해 입력데이터를 `Dataset`으로 변환해 처리합니다.  
`Dataset`은 순회가능한 객체(iterable object)입니다.  
[from_tensor_slices](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) 메서드를 이용하면 입력 텐서를 슬라이싱해서 `Dataset`을 만듭니다.  
![](https://drive.google.com/thumbnail?id=1c_shLACT0K4xWVN7MMZ3SOIPo9OTQBzV&sz=s4000)

In [ ]:
import numpy as np
import tensorflow as tf

dataset = np.arange(100).reshape(20,5)
print(dataset)

dataset = tf.data.Dataset.from_tensor_slices(dataset)

for data in dataset:
    print(data)

[batch](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch) 메서드를 써서 배치 단위로 묶을 수 있습니다.

In [ ]:
batched_dataset = dataset.batch(4)

for batch in batched_dataset:
    print(batch)

[tf.keras.utils.image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory)을 사용하면 디렉토리로부터 데이터를 읽어들여 `Dataset`을 만듭니다.  
- `image_size=(180, 180)` : 해상도가 제 각각 이었는데 180$\times$180으로 통일해줍니다. 그래야 신경망을 구성할 수 있겠지요.  
- `batch_size=32` : 입력 데이터셋을 32개의 배치단위로 묶습니다.
- `shuffle=True` : True가 디폴트 설정인데 입력데이터를 랜덤하게 섞습니다.
- `labels='inferred'`: inferred가 디폴트 설정인데 디렉토리 이름의 알파벳 순서에 따라 텐서플로우가 라벨을 0부터 차례대로 붙여줍니다. train, validation, test 디렉토리 밑에 모두 cat, dog 디렉토리가 있기 때문에 고양이 사진은 0, 강아지 사진은 1로 라벨을 붙여줍니다.

In [ ]:
import pathlib
from tensorflow.keras.utils import image_dataset_from_directory

base_dir = pathlib.Path("cats_vs_dogs_small")

train_dataset = image_dataset_from_directory(
    base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

iter로 묶은후 next를 적용하면 첫번째 배치묶음을 출력할 수 있습니다.  
텐서와 라벨의 튜플입니다.

In [ ]:
iterator = iter(train_dataset)
batch_1 = next(iterator)
print(batch_1)

두번째 배치묶음입니다.

In [ ]:
batch_2 = next(iterator)
print(batch_2)

모든 라벨을 출력해봤습니다.

In [ ]:
for data in train_dataset:
    print(data[1])

**[실습3] (10분) 첫번재 배치묶음을 8$\times$4 모아찍기로 출력하시오. 이미지 밑에 라벨을 출력하시오..**

# 적은 데이터로 학습

합성곱 신경망을 구성하겠습니다.  
데이터가 복잡한 만큼 전보다 더 Deep합니다.  
tf.keras.layers.MaxPooling2D는 tf.keras.layers.MaxPool2D와 완전히 동일합니다.

In [ ]:
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Rescaling, Flatten, Dense

inputs = keras.Input(shape=(180, 180, 3))
x = Rescaling(1./255)(inputs)
x = Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = Flatten()(x)
outputs = Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

콜백으로 자동저장을 지정합니다.  
위에서 만든`Dataset`으로 학습과 검증을 합니다.  
배치크기는 지정할 필요 없습니다.

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = keras.callbacks.ModelCheckpoint(
    filepath="convnet_from_scratch.keras",
    save_best_only=True,
    monitor="val_loss")

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

데이터가 적어서 과적합이 심하게 일어납니다.

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

최고 성능일때 자동저장된 모델을 불러왔습니다.  
찍어도 50%라는 점을 생각하면 기대보다 낮은 정확도입니다.  
데이터의 복잡도에 비해 훈련 데이터 수가 너무 적어서인 듯 합니다.

In [ ]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"테스트 정확도: {test_acc:.3f}")

# 데이터 증강

모델을 구성하고 하이퍼 파라미터를 튜닝하는 일보다 현실에서는 충분한 데이터를 확보하는게 더 어려울 수 있습니다.  
적은 데이터로부터 변형을 통해 데이터 양을 인위적으로 늘리는 기법을 데이터 증강 (data augmentation)이라고 합니다.  
![](https://drive.google.com/thumbnail?id=1kjG_oa9qvarB8UTX63LkSia9_a-jxDPZ&sz=s4000)

---
이미지 데이터는 대칭, 회전, 확대, 축소등을 통해 데이터를 인위적으로 늘릴 수 있습니다.  
대칭은 [tf.keras.layers.RandomFlip](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomFlip)으로, 회전은 [tf.keras.layers.RandomRotation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomRotation)으로, 확대 축소는 [tf.keras.layers.RandomZoom](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomZoom)으로 구현되어 있습니다.  
Squential 클래스로 묶겠습니다.
- `RandomFlip("horizontal")` : 랜덤하게 좌우대칭
- `RandomRotation(0.1)` : 랜덤하게 -0.1×360° ~ 0.1×360°만큼 회전
- `RandomZoom(0.2)` : 상하로 랜덤하게 -20% ~ 20% 확대

In [ ]:
from keras.layers import RandomFlip, RandomRotation, RandomZoom

data_augmentation = keras.Sequential(
    [RandomFlip("horizontal"),
     RandomRotation(0.1),
     RandomZoom(0.2)])

첫번째 배치묶음의 첫번째 데이터에 대해서 증강을 했습니다.

In [ ]:
img = batch_1[0][0]

plt.imshow(img/255)
plt.xticks([])
plt.yticks([])
plt.show()

plt.figure(figsize=(12, 12))
for i in range(25):
    augmented_img = data_augmentation(img[np.newaxis,:,:,:])
    plt.subplot(5, 5, i + 1)
    plt.imshow(augmented_img[0]/255)
    plt.xticks([])
    plt.yticks([])
plt.show()

**[실습4] (10분) (i) MNIST 첫번째 훈련 데이터에 대하여 랜덤하게 -0.1$\times$360° ~ 0.1$\times$360°만큼 회전한 이미지 25장을 5$\times$5 모아찍기로 출력하시오.**

In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

**(ii) MNIST 첫번째 훈련 데이터에 대하여 상하좌우로 랜덤하게 20프로 이내로 평행이동한 이미지 25장을 5$\times$5 모아찍기로 출력하시오.**

# 데이터 증강을 통한 학습

첫번째 층에서 입력 데이터를 변형합니다.  
에퍽 수만큼 동일한 데이터가 반복해서 들어올텐데 들어올때마다 변형해서 적은 훈련 데이터 개수를 보완해줍니다.  
과적합을 피하기 위해 Dropout층을 추가했습니다.

In [ ]:
from keras.layers import Dropout

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = Rescaling(1./255)(x)
x = Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [keras.callbacks.ModelCheckpoint(
    filepath="convnet_from_scratch_with_augmentation.keras",
    save_best_only=True,
    monitor="val_loss")]

history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

데이터 증강을 통해 대략 70% → 80%의 성능 향상을 이뤄냈습니다.

In [ ]:
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"테스트 정확도: {test_acc:.3f}")